<a href="https://colab.research.google.com/github/CarolinaNovaesGN/projeto-dataset-markt/blob/main/pysparkSQL_mini_projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instalando e confifurando spark/sparkSQL**

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.window import Window as W
spark = (SparkSession.builder
        .master("local")
        .appName("mini-projeto-sql")
        .config("spark.ui.port","4050")
        .getOrCreate())

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 49.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=c05b5644503ec503f0d6629dc6ba65b780e5589949a498a401d23c324fd9a1b4
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


**Criando a tabela temporária para as consultas em SQL**

In [ ]:
df = spark.read.format("csv").option("inferSchema","true").option("header","true").option("sep",",").load("/content/pysp_df_geral2.csv").createOrReplaceTempView("dados_compras_table")

**Visualizando a tabela**

In [ ]:
query='''
SELECT *
FROM dados_compras_table
LIMIT 10
'''
spark.sql(query).toPandas()

,nmr_linha,id,ano_nascimento,educacao,status_conjugal,renda,criancas_casa,adolescentes_casa,data_cliente,vinhos,frutas,carne_produtos,peixe_produtos,doces_produtos,ouro_produtos,qtd_oferta_compras,qtd_web_compras,qtd_produtos_catalogo,qtd_produtos_armazenados,qtd_web_visitantes_mensal,reclamacoes,total_produtos,total_compras,ano
0,0,5524,1957,graduacao,Solteiro,58138.0,0,0,2012-09-04,635,88,546,172,88,88,3,8,10,4,7,0,2724,33,2012
1,1,2174,1954,graduacao,Solteiro,46344.0,1,1,2014-03-08,11,1,6,2,1,6,2,1,1,2,5,0,54,9,2014
2,4,5324,1981,doutorado,Casado,58293.0,1,0,2014-01-19,173,43,118,46,27,15,5,5,3,6,5,0,645,30,2014
3,6,965,1971,graduacao,Divorciado,55635.0,0,1,2012-11-13,235,65,164,50,49,27,4,7,3,7,6,0,900,33,2012
4,7,6177,1985,doutorado,Casado,33454.0,1,0,2013-05-08,76,10,56,3,1,23,2,4,0,4,8,0,261,18,2013
5,11,387,1976,basico,Casado,7500.0,0,0,2012-11-13,6,16,11,11,1,16,1,2,0,3,8,0,126,9,2012
6,12,2125,1959,graduacao,Divorciado,63033.0,0,0,2013-11-15,194,61,480,225,112,30,1,3,4,8,2,0,2577,12,2013
7,13,8180,1952,mestre,Divorciado,59354.0,1,1,2013-11-15,233,2,53,3,5,14,3,6,1,5,6,0,243,27,2013
8,14,2569,1987,graduacao,Casado,17323.0,0,0,2012-10-10,3,14,17,6,1,5,1,1,0,3,8,0,96,6,2012
9,15,2114,1946,doutorado,Solteiro,82800.0,0,0,2012-11-24,1006,22,115,59,68,45,1,7,6,12,3,0,915,24,2012


**1 - Contando quantas reclamações foram reportadas:**

In [ ]:
query='''
SELECT COUNT(reclamacoes) 
AS qts_reclamacoes 
FROM dados_compras_table 
WHERE reclamacoes != 0
'''
spark.sql(query).toPandas()

,qts_reclamacoes
0,15


comparando com a quantidade de linhas da tabela

In [ ]:
query='''
SELECT COUNT(id) 
FROM dados_compras_table
'''
spark.sql(query).toPandas()

,count(id)
0,1634


Apenas 0,9 % dos clientes fizeram uma reclamação

**2 - Qual estado civil consome mais?**

In [ ]:
query='''
SELECT status_conjugal,
sum(total_produtos)  as qtd
FROM dados_compras_table 
group by status_conjugal
order by qtd desc
'''
spark.sql(query).toPandas()

,status_conjugal,qtd
0,Casado,705585
1,Solteiro,427116
2,Divorciado,185586
3,Viuvo,77205


Pessoas casadas são o maior público da loja.

**3 - Cliente com a maior renda e quantos produtos ele adquiriu.**

In [ ]:
query = '''
SELECT MAX(renda) 
AS maior_renda 
FROM dados_compras_table
'''
spark.sql(query).toPandas()

,maior_renda
0,160803.0


In [ ]:
query = '''
SELECT * 
FROM dados_compras_table 
WHERE renda = 160803 
'''
spark.sql(query).toPandas()

,nmr_linha,id,ano_nascimento,educacao,status_conjugal,renda,criancas_casa,adolescentes_casa,data_cliente,vinhos,frutas,carne_produtos,peixe_produtos,doces_produtos,ouro_produtos,qtd_oferta_compras,qtd_web_compras,qtd_produtos_catalogo,qtd_produtos_armazenados,qtd_web_visitantes_mensal,reclamacoes,total_produtos,total_compras,ano
0,687,1501,1982,doutorado,Casado,160803.0,0,0,2012-08-04,55,16,1622,17,3,4,15,0,28,1,0,0,5025,45,2012


**4 - IDs das 20 maiores compras**

In [ ]:


query='''
SELECT id, 
total_compras, 
ano 
FROM dados_compras_table 
ORDER BY total_compras DESC LIMIT 20
'''


spark.sql(query).toPandas()

,id,total_compras,ano
0,4619,81,2014
1,10311,75,2013
2,6237,69,2013
3,3594,63,2012
4,8581,57,2013
5,531,57,2012
6,4478,57,2013
7,5207,57,2012
8,2408,57,2012
9,3678,57,2012


Mais da metade das 20 maiores compras foram feitas por pessoas com nível de escolaridade: graduação


**5-10 Maiores compras em 2014**

---



In [ ]:
query='''
SELECT 
 RANK() OVER (PARTITION BY ano ORDER BY total_compras Desc) as Rank,

 ano,
 total_compras,
 id as id
 
FROM dados_compras_table
WHERE ano=2014
ORDER BY total_compras Desc  LIMIT 10
'''
spark.sql(query).toPandas()

,Rank,ano,total_compras,id
0,1,2014,81,4619
1,2,2014,54,6892
2,3,2014,48,5136
3,4,2014,45,8475
4,4,2014,45,9931
5,4,2014,45,6862
6,7,2014,42,1584
7,8,2014,39,3924
8,8,2014,39,10846
9,8,2014,39,6722


In [ ]:
query='''
SELECT 
id,count(id) as quantidade
FROM dados_compras_table
group by id
having quantidade=1
'''
spark.sql(query).limit(5).toPandas()

,id,quantidade
0,6336,1
1,5300,1
2,5156,1
3,8086,1
4,6357,1


**6- Quantos produtos são adquiridos por compra em média?**

In [ ]:
query='''
SELECT
total_produtos,
total_compras,
total_produtos/total_compras as media
FROM dados_compras_table

LIMIT 5
'''
spark.sql(query).toPandas()

,total_produtos,total_compras,media
0,2724,33,82.545455
1,54,9,6.000000
2,645,30,21.500000
3,900,33,27.272727
4,261,18,14.500000


**7- Quem compra mais? Quem tem criança ou não?**

In [ ]:
query='''
SELECT criancas_casa,
sum(total_produtos)  as qtd
FROM dados_compras_table 
group by criancas_casa
order by qtd desc
'''
spark.sql(query).toPandas()

,criancas_casa,qtd
0,0,1198683
1,1,190068
2,2,6741


**8 - Maiores compras de ouro**

In [ ]:
query='''
SELECT id,
ouro_produtos
FROM dados_compras_table 
order by ouro_produtos desc
LIMIT 5
'''
spark.sql(query).toPandas()

,id,ouro_produtos
0,10311,321
1,6237,291
2,4246,262
3,10936,249
4,6721,248


**9- Qual estado civil consome mais carne?**

In [ ]:
query='''
SELECT status_conjugal,
sum(carne_produtos)  as qtd
FROM dados_compras_table 
group by status_conjugal
order by qtd desc
'''
spark.sql(query).toPandas()

,status_conjugal,qtd
0,Casado,137888
1,Solteiro,87059
2,Divorciado,34840
3,Viuvo,14085


**Só tem valores 0 e 1 na coluna reclamações?**

In [ ]:
query='''
SELECT COUNT (DISTINCT reclamacoes)
FROM dados_compras_table 
'''
spark.sql(query).toPandas()

,count(DISTINCT reclamacoes)
0,2
